<a href="https://colab.research.google.com/github/Tejaswini170104/DA6401-A2/blob/main/PartB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Question 01**

### (a) Adjusting Input Image Dimensions for Pretrained Models

- **ImageNet pretrained models** (e.g., ResNet, VGG, EfficientNet) expect input images to have a resolution of **224×224 pixels**.
- **iNaturalist dataset images** may have different sizes and aspect ratios.
- To ensure compatibility with the pretrained model:
  - **Resize** all images to match the input size expected by the model.
  - Use the `transforms.Resize()` function in PyTorch to resize images.
  - Additional cropping using `transforms.CenterCrop()` can be applied to focus on the central content of the image.
- Example of resizing:

  ```python
  transforms.Resize((224, 224))  # Resize to match input size expected by most pretrained models
  ```




### (b) Replacing the Output Layer for a 10-Class Classification Task

- **Pretrained models** are designed for **ImageNet’s 1000-class classification**.
- The iNaturalist dataset has only **10 classes**, so the output layer must be adjusted.
- **Modification**:
  - Replace the last fully connected (FC) layer to output 10 logits (one for each class).
  - This ensures the model outputs the correct number of predictions for the iNaturalist task.
- Example modification for ResNet50:

  ```python
  from torchvision import models
  import torch.nn as nn

  model = models.resnet50(pretrained=True)
  num_features = model.fc.in_features
  model.fc = nn.Linear(num_features, 10)  # Update final layer for 10-class output
  ```



**Question 02**

### Strategies to Make Fine-Tuning Tractable

- **Freezing All Layers Except the Last Layer**:
  - In this approach, all the layers of the model are frozen, meaning their weights remain unchanged during training. Only the final fully connected (FC) layer is trainable, allowing the model to adapt to the new task without requiring full retraining.
  - This approach leverages the pretrained features in the earlier layers, which are generally useful for a wide range of tasks, while only training the final layer to match the specific 10-class task.

  Example:
  ```python
  for param in model.parameters():
      param.requires_grad = False  # Freeze all layers

  # Only the last layer will be updated
  model.fc.requires_grad = True  # Unfreeze the last FC layer
  ```

- **Freezing Layers Up to the k-th Layer**:
  - In this strategy, the first few layers are frozen, and only the layers after a certain point (e.g., the k-th layer) are allowed to be updated. The idea here is to freeze the early layers which learn basic features (like edges and textures) that are likely transferable across different tasks, while fine-tuning the later layers to capture task-specific details.
  - This approach is a middle ground between freezing all layers and training the entire model.

  Example:
  ```python
  for i, param in enumerate(model.parameters()):
      if i < k:
          param.requires_grad = False  # Freeze layers up to k-th layer
      else:
          param.requires_grad = True  # Unfreeze layers after k-th layer
  ```

- **Fine-Tuning Only the Last Few Layers**:
  - Another approach is to **fine-tune only the last few layers**, such as the final few convolutional or dense layers. The idea behind this strategy is to allow the model to adapt to the specific characteristics of the new dataset while retaining the powerful features learned by the earlier layers.
  - In this case, the number of layers to fine-tune can be adjusted based on computational constraints and the task's complexity.

  Example:
  ```python
  # Freezing early layers and fine-tuning the last few layers
  for param in model.parameters():
      param.requires_grad = False  # Freeze all layers

  # Unfreeze the last 2 layers
  for param in model.layer4.parameters():
      param.requires_grad = True
  ```
